### Configuration

In [6]:
import os
import numpy as np
import pandas as pd

import mne
from tqdm import tqdm

import utils__config

In [7]:
os.chdir(utils__config.working_directory)
os.getcwd()

'Z:\\Layton\\Sleep_083023'

### Parameters

In [8]:
fif_path = 'Cache/Subject01/Feb02/S01_Feb02_256hz.fif'
dict_path = 'Data/Subject01/Feb02/S01_dictionary.xlsx'

sw_path = 'Cache/Subject01/Feb02/S01_SW.csv'
kc_path = 'Cache/Subject01/Feb02/S01_KC.csv'
save_path = 'Cache/Subject01/Feb02/S01_event_epochs.csv'

In [9]:
sampling_freq = 256

### Load Raw data

In [10]:
# Load LFP data
raw = mne.io.read_raw_fif(fif_path, preload = True, verbose = None)

# Record the first sample (which is not 0 since the Raw
# file was cropped from the original); you will need this
# to appropriately select the sleep event sample number
first_raw_sample = raw.first_samp

# Remove unnecessary channels
#dictionary = pd.read_excel(dict_path)
#dictionary = dictionary[(dictionary['type'] == 'macro') & (dictionary['status'] == 'accept')]['name'].tolist()
#raw.pick_channels(ch_names = dictionary)

# Create a dummy numpy event array and MNE info object
# and use them to create an empty dummy Raw channel
events_info = mne.create_info(ch_names = ['events'], 
                              sfreq = raw.info['sfreq'], 
                              ch_types = ['stim'])

empty_events = np.zeros((1, len(raw.times)))

events_channel = mne.io.RawArray(data = empty_events, 
                                 info = events_info,
                                 first_samp = first_raw_sample)

Opening raw data file Cache/Subject01/Feb02/S01_Feb02_256hz.fif...


C:\Users\lal85\AppData\Local\Temp\2\ipykernel_7976\2216903412.py:2: RuntimeWarning: This filename (Cache/Subject01/Feb02/S01_Feb02_256hz.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fif_path, preload = True, verbose = None)


Isotrak not found
    Range : 0 ... 1843573 =      0.000 ...  7201.457 secs
Ready.
Reading 0 ... 1843573  =      0.000 ...  7201.457 secs...
Creating RawArray with float64 data, n_channels=1, n_times=1843574
    Range : 0 ... 1843573 =      0.000 ...  7201.457 secs
Ready.


### Slow Waves

In [11]:
# Load detected slow waves
sw_times = pd.read_csv(sw_path)
sw_times = sw_times[['Channel', 'Start', 'End', 'Duration', 'NegPeak', 'MidCrossing', 'PosPeak']]
sw_times.columns = ['channel', 'start', 'end', 'duration', 'neg_peak', 'mid_cross', 'pos_peak']

# Find MNE sample times for when SW's were present
sw_times['sample'] = (sw_times.neg_peak * sampling_freq) + first_raw_sample
sw_times['sample'] = sw_times['sample'].round(0).astype('int64')

# Create a numpy array formatted according to MNE requirements
# (please note that the dummy_0 column is just a spacer and has 
#  no meaning except internally to MNE, which expects 0's; 
#  however, the event_id column should contain the integer that
#  corresponds to the relevant event in your event dictionary)
sw_times['dummy_0'] = 0
sw_times['event_id'] = 1

### K-Complexes

In [12]:
# Load detected slow waves
kc_times = pd.read_csv(kc_path)
kc_times = kc_times[['Channel', 'Start', 'End', 'Duration', 'NegPeak', 'MidCrossing', 'PosPeak']]
kc_times.columns = ['channel', 'start', 'end', 'duration', 'neg_peak', 'mid_cross', 'pos_peak']

# Find MNE sample times for when SW's were present
kc_times['sample'] = (kc_times.neg_peak * sampling_freq) + first_raw_sample
kc_times['sample'] = kc_times['sample'].round(0).astype('int64')

# Create a numpy array formatted according to MNE requirements
# (please note that the dummy_0 column is just a spacer and has 
#  no meaning except internally to MNE, which expects 0's; 
#  however, the event_id column should contain the integer that
#  corresponds to the relevant event in your event dictionary)
kc_times['dummy_0'] = 0
kc_times['event_id'] = 2

### Mark events and export by channel

In [13]:
event_frame = pd.DataFrame()

# Loop through only those channels with detected SW or KC
sw_kc_channels = pd.concat([sw_times['channel'], kc_times['channel']]).unique()

for channel in tqdm(sw_kc_channels):

    # Create a channel-specific Raw object
    temp_raw = raw.copy().pick_channels(ch_names = [channel])

    # Create channel-specific Event numpy arrays
    chan_sw = np.array(sw_times[sw_times.channel == channel][['sample', 'dummy_0', 'event_id']])
    chan_kc = np.array(kc_times[kc_times.channel == channel][['sample', 'dummy_0', 'event_id']])

    # Add the empty dummy channel to the data
    temp_raw.add_channels([events_channel], force_update_info = True)

    # Now update the dummy events ('stim') channel with the 
    # event times and then convert them to MNE events
    event_dictionary = {'slow_wave' : 1}
    
    temp_raw.add_events(chan_sw, 'events')

    # If any K-Complexes were detected, overwrite above
    if (len(chan_kc) > 0):

        event_dictionary = {'slow_wave' : 1,
                            'k_complex' : 2}
        
        temp_raw.add_events(chan_kc, 'events')

    # Find events
    events = mne.find_events(raw = temp_raw, 
                             shortest_event = 1, # include events of length 1 sample 
                             initial_event = True) # include events occuring at sample 0

    # Create an MNE Epochs object using the modified MNE Raw object
    epochs = mne.Epochs(temp_raw, 
                        preload = True, 
                        events = events, 
                        event_id = event_dictionary, 
                        baseline = None, 
                        verbose = True, 
                        tmin = -2, 
                        tmax = 2, 
                        decim = 1)

    # Drop the event channel before exporting data
    epochs = epochs.drop_channels(['events'])

    # Format the Epochs object into a pandas dataframe
    epochs = epochs.to_data_frame(scalings = dict(seeg = 1e6, eeg = 1e6))
    epochs = epochs.set_index(['time', 'condition', 'epoch']).stack().reset_index()
    epochs.columns = ['time', 'condition', 'epoch', 'channel', 'value']

    # Average sleep events within channel (optional)
    epochs.drop(columns = ['epoch'], inplace = True)
    epochs = epochs.groupby(['time', 'condition', 'channel']).mean().reset_index()

    # Get data and annotate event type (SW/KC)
    event_frame = pd.concat([event_frame, epochs])

  0%|          | 0/34 [00:00<?, ?it/s]

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
247 events found
Event IDs: [1]
Not setting metadata
247 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 247 events and 1025 original time points ...
0 bad epochs dropped


  3%|▎         | 1/34 [00:00<00:14,  2.24it/s]

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
237 events found
Event IDs: [1]
Not setting metadata
237 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 237 events and 1025 original time points ...
0 bad epochs dropped


  6%|▌         | 2/34 [00:00<00:12,  2.47it/s]

...

 94%|█████████▍| 32/34 [00:17<00:01,  1.60it/s]

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
1284 events found
Event IDs: [1 2 3]
Not setting metadata
1283 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1283 events and 1025 original time points ...
0 bad epochs dropped


 97%|█████████▋| 33/34 [00:18<00:00,  1.46it/s]

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
1357 events found
Event IDs: [1 2]
Not setting metadata
1357 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1357 events and 1025 original time points ...
0 bad epochs dropped


100%|██████████| 34/34 [00:19<00:00,  1.76it/s]


In [14]:
event_frame.to_csv(save_path)